# gpt-5-mini - 속도,비용,품질 밸런스
| 모델             | 특징            | RAG 상담용  |
| -------------- | ------------- | -------- |
| gpt-5          | 최고 품질, 느리고 비쌈 | ❌ 과함     |
| **gpt-5-mini** | 빠름 + 정확       | ✅ **최적** |
| gpt-5-nano     | 매우 빠름, 요약 위주  | ⚠️ 답변 얕음 |


In [2]:
!pip install -U \
  python-dotenv \
  langchain-community \
  langchain-text-splitters \
  langchain-openai \
  faiss-cpu \
  pypdf


  Using cached python_dotenv-1.2.1-py3-none-any.whl.metadata (25 kB)
  Using cached tenacity-9.1.2-py3-none-any.whl.metadata (1.2 kB)
  Using cached langsmith-0.4.59-py3-none-any.whl.metadata (15 kB)
  Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl.metadata (5.9 kB)
  Using cached aiosignal-1.4.0-py3-none-any.whl.metadata (3.7 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached uuid_utils-0.12.0-cp39-abi3-win_amd64.whl.metadata (1.1 kB)
  Using cached requests_toolbelt-1.0.0-py2.py3-none-any.whl.metadata (14 kB)
  Using cached mypy_extensions-1.1.0-py3-none-any.whl.metadata (1.1 kB)
Using cached python_dotenv-1.2.1-py3-none-any.whl (21 kB)
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.5 MB ? eta -:--:--
   -------- ------------------------------- 0.5/2.5 MB 1.1 MB/s eta 0:00:02
   -------- ------------------------------- 0.5/2.5 MB 1.1 MB/s eta 0:00:02
   ---------------- 

In [5]:
# ======================================
# 1. 환경 변수 로드 (.env)
# ======================================
from dotenv import load_dotenv
import os

load_dotenv()
assert os.getenv("OPENAI_API_KEY"), "OPENAI_API_KEY가 .env에 없습니다"

# ======================================
# 2. PDF 로딩
# ======================================
import time
from langchain_community.document_loaders import PyPDFLoader

file_path = r"E:\ai\Bidding-Mate\data\raw\file\고려대학교_차세대 포털·학사 정보시스템 구축사업.pdf"

t0 = time.time()
loader = PyPDFLoader(file_path)
docs = loader.load()
print("📄 PDF 로딩:", round(time.time() - t0, 2), "초")

# ======================================
# 3. 문서 쪼개기
# ======================================
from langchain_text_splitters import RecursiveCharacterTextSplitter

t0 = time.time()
splitter = RecursiveCharacterTextSplitter(
    chunk_size=700,
    chunk_overlap=150,
)
splits = splitter.split_documents(docs)
print("✂️ 문서 분할:", round(time.time() - t0, 2), "초")

# ======================================
# 4. 🚀 임베딩 (OpenAI API)
# ======================================
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

t0 = time.time()
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small"
)

vectorstore = FAISS.from_documents(splits, embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})
print("🧠 임베딩:", round(time.time() - t0, 2), "초")

# ======================================
# 5. LLM (OpenAI API)
# ======================================
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-5-mini",   # 속도/비용 최적
    temperature=0.2,
)

# ======================================
# 6. 프롬프트
# ======================================
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template("""
아래 [맥락]을 근거로 질문에 답하세요.
맥락에 없는 내용은 추측하지 말고 "문서에 없음"이라고 답하세요.
반드시 한국어로 답하세요.

[맥락]
{context}

[질문]
{question}
""")

# ======================================
# 7. RAG 체인
# ======================================
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {
        "context": retriever | format_docs,
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
    | StrOutputParser()
)

# ======================================
# 8. 테스트
# ======================================
question = """- 고려대학교 차세대 포털 시스템 사업이랑 광주과학기술원의 학사 시스템 기능개선 사업을 비교해 줄래?
    - 고려대학교랑 광주과학기술원 각각 응답 시간에 대한 요구사항이 있나? 문서를 기반으로 정확하게 답변해 줘."""

t0 = time.time()
answer = rag_chain.invoke(question)
print("\n⏱️ 응답 시간:", round(time.time() - t0, 2), "초")
print("\n📌 답변:\n", answer)


📄 PDF 로딩: 19.33 초
✂️ 문서 분할: 0.02 초
🧠 임베딩: 7.96 초

⏱️ 응답 시간: 8.46 초

📌 답변:
 요청하신 비교는 제공된 문서(맥락)를 기준으로만 답변드립니다.

요약
- 고려대학교 차세대 포털·학사 정보시스템 구축 사업: 문서에 응답시간(응답 속도, SLA 등)에 대한 요구사항이 기재되어 있지 않습니다. (사업명, 기간, 예산, 지급계획, 입찰방법, 무상유지보수기간 등은 나와 있으나 응답시간 관련 명시는 없음)
- 광주과학기술원(학사 시스템 기능개선 사업): 본 맥락에 관련 내용이 전혀 포함되어 있지 않습니다. 따라서 응답시간 요구사항 여부를 판단할 수 없습니다. -> 문서에 없음

원하시면 고려대학교 문서에서 응답시간 외에 비교 가능한 다른 항목(사업기간, 예산, 유지보수 등)을 정리해 드리거나, 광주과학기술원 관련 문서를 제공해 주시면 두 사업의 응답시간 요구사항을 정확히 비교해 드리겠습니다.


# 문서 질문

In [6]:
question = """- 고려대학교 차세대 포털 시스템 사업관련된 문서에 대한 정보를 알려줄래? 문서내 요소들도 어떻게 되어있니? 사업의 목적은 뭐니? 문서를 기반으로 정확하게 답변해 줘."""

t0 = time.time()
answer = rag_chain.invoke(question)
print("\n⏱️ 응답 시간:", round(time.time() - t0, 2), "초")
print("\n📌 답변:\n", answer)


⏱️ 응답 시간: 23.18 초

📌 답변:
 요청하신 문서는 고려대학교의 "차세대 포털·학사 정보시스템 구축 사업" 관련 제안/요청 자료(제안요청서)이며, 문서에 명시된 내용과 그 항목은 다음과 같습니다.

문서 기본정보
- 문서명(사업명): 고려대학교 차세대 포털·학사 정보시스템 구축 사업
- 문서 형태: 제안요청서(제안내용 설명을 위한 배포자료, 무단복제 금지)
- 작성/배포일: 2024. 7. 01

문서 내 주요 요소(명시된 항목과 값)
1. 사업개요
- 사업기간: 계약일로부터 24개월 이내
- 무상유지보수기간: 사업종료일로부터 12개월
- 사업예산: 11,270,000,000원 (부가가치세 포함, 3년 분할 지급)
  - 지급비율: 2024학년도 약 30%, 2025학년도 약 40%, 2026학년도 약 30%
- 입찰 및 계약 방법: 제한 경쟁 입찰(협상에 의한 계약)
- 사업특성: 3개년에 걸친 다년 사업(대가도 3개년에 걸쳐 분할 지급)

2. 추진 배경 및 필요성(사업 목적과 연관된 항목)
- 교육 환경의 급격한 변화 대응: 학령인구 감소, 코로나 팬데믹에 따른 교육환경 변화, 인공지능 활용 등 사회·기술 변화 대응 필요
- 분산된 시스템 및 데이터의 통합 필요: 노후 학사 시스템과 업무별 분산된 정보화로 인해 시스템 통합 요구 증가
- 데이터 기반 대학경영 지원 개선 필요: 분절된 정보시스템 간 정보연계 미흡 및 정보화 표준 부재로 경영현황 파악·의사결정에 한계 발생

3. 부속서식 예시(문서에 포함된 양식)
- [서식 25] 보안서약서(용역업체 직원용): 외부업체 직원의 보안서약 적용, 주요 조항 요약(정보의 업무범위 이용, 정보자산 안전관리, 무단누설 금지, 허가 없는 접근 금지, 원격지원 시 사전통보·승인, 정보자산 외부 반출 금지, 자원(ID/비밀번호 등) 관리 등)

문서에 기반한 사업의 목적(요약)
- 급격한 교육·기술환경 변화에 대응하는 것
- 분산된 학사·포털 시스템과 데이터를 통합하여 효율화하는 것
- 정보연계와 표준화를 통

# 환각 질문

In [7]:
t0 = time.time()
answer = rag_chain.invoke("저녁 메뉴 추천")
print("\n⏱️ 응답 시간:", round(time.time() - t0, 2), "초")
print("\n📌 답변:\n", answer)


⏱️ 응답 시간: 4.32 초

📌 답변:
 문서에 없음
